In [1]:
import pandas as pd
import numpy as np

from pythainlp import word_tokenize
import re

import tensorflow as tf

from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras import *
from keras.layers import *

from sklearn.linear_model import LogisticRegression

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', 
    patience=8, 
    min_delta=0.001, 
    mode='max'
)

In [2]:
df = pd.read_csv('main_suicidal_data.csv')

# Data Prep

In [3]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|[^\s()]*?\)|)+(?:[^\s()]*?\)||[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def process_text(text):
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)

    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    return res

In [4]:
DOMAIN_DICT = {
      0 : 'Level 1',
      1 : 'Level 2',
      2 : 'Level 3',
      3 : 'Level 4',
      4 : 'Level 5',
      5 : 'Other',
  }

In [5]:
df['Tweet'] = df['Tweet'].map(lambda x: '|'.join(process_text(x)))

In [6]:
ws_tfidf = TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 2), sublinear_tf=True)
ws_vec = ws_tfidf.fit_transform(df['Tweet']).toarray()
X = ws_vec

In [7]:
y = df['Label (Specialist)']

le = LabelEncoder()
le.fit(y)
y = le.transform(y)

# Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Logistic Reg

In [9]:
model = LogisticRegression()
model.fit(X_train,y_train)

/Users/yo/miniforge3/envs/mlp/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
y_pred = model.predict(X_test)
y_true = np.array(y_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.96      0.83       286
           1       0.46      0.34      0.39        38
           2       0.73      0.28      0.40        29
           3       0.73      0.20      0.31        40
           4       0.57      0.56      0.57        57
           5       0.69      0.39      0.50        64

    accuracy                           0.70       514
   macro avg       0.65      0.45      0.50       514
weighted avg       0.69      0.70      0.66       514



# Random Forest Classifier

In [11]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier() 
clf.fit(X_train, y_train)
y_pred_rf_not_tuning = clf.predict(X_test)

In [12]:
print(classification_report(y_test, y_pred_rf_not_tuning))

              precision    recall  f1-score   support

           0       0.73      0.93      0.82       286
           1       0.39      0.34      0.37        38
           2       0.69      0.38      0.49        29
           3       0.44      0.17      0.25        40
           4       0.59      0.56      0.58        57
           5       0.63      0.30      0.40        64

    accuracy                           0.68       514
   macro avg       0.58      0.45      0.48       514
weighted avg       0.65      0.68      0.64       514



# KNN

In [13]:
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2 )  
classifier.fit(X_train, y_train)  

knn_y_pred = classifier.predict(X_test)
print(classification_report(y_true, knn_y_pred))

              precision    recall  f1-score   support

           0       0.67      0.92      0.78       286
           1       0.32      0.16      0.21        38
           2       0.54      0.45      0.49        29
           3       0.62      0.25      0.36        40
           4       0.61      0.44      0.51        57
           5       0.59      0.20      0.30        64

    accuracy                           0.64       514
   macro avg       0.56      0.40      0.44       514
weighted avg       0.62      0.64      0.60       514



# SGD Classifier

In [15]:
from sklearn.linear_model import SGDClassifier

In [18]:
SGDclf = SGDClassifier(max_iter=1000, tol=0.01)
SGDclf.fit(X_train,y_train)

SGDclf_pred = SGDclf.predict(X_test)
print(classification_report(y_true, SGDclf_pred))

              precision    recall  f1-score   support

           0       0.76      0.90      0.82       286
           1       0.43      0.39      0.41        38
           2       0.58      0.52      0.55        29
           3       0.50      0.25      0.33        40
           4       0.58      0.49      0.53        57
           5       0.60      0.44      0.50        64

    accuracy                           0.69       514
   macro avg       0.57      0.50      0.53       514
weighted avg       0.67      0.69      0.67       514

